# Set-Up

## Installation and Imports

In [9]:
!pip install tetgen
!apt-get install -qq xvfb
!pip install pyvista panel -q
!pip install -q piglet pyvirtualdisplay
!pip install pygmsh
!pip install tqdm
!pip uninstall -y h5py
!pip install h5py==2.9.0

     |████████████████████████████████| 1.9 MB 4.1 MB/s eta 0:00:01     |████████████████████████▌       | 1.5 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 88.3 MB 3.9 kB/s  eta 0:00:01    |████▎                           | 11.8 MB 8.6 MB/s eta 0:00:09     |█████                           | 13.6 MB 8.6 MB/s eta 0:00:09     |█████▊                          | 15.8 MB 8.6 MB/s eta 0:00:09     |██████                          | 16.3 MB 8.6 MB/s eta 0:00:09     |█████████▎                      | 25.6 MB 7.3 MB/s eta 0:00:09     |██████████████▊                 | 40.7 MB 5.0 MB/s eta 0:00:10     |████████████████▏               | 44.7 MB 5.0 MB/s eta 0:00:09     |████████████████▎               | 45.0 MB 5.0 MB/s eta 0:00:09     |██████████████████▍             | 50.8 MB 5.9 MB/s eta 0:00:07     |█████████████████████           | 57.9 MB 

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(600, 400))
display.start()

In [165]:
# Fenics imports:
import dolfinx
import dolfinx.io
import dolfinx.plot
import dolfinx.geometry
from dolfinx.cpp.mesh import CellType

# Numerics imports:
from petsc4py import PETSc
import numpy as np

# Mesh imports:
import gmsh
import pygmsh
import meshio
import ufl
from mpi4py import MPI
import tetgen

# Visualisation imports:
import pyvista

# Misc imports:
from math import sin, cos, pi, ceil, floor
import os

from tqdm import tqdm

## Mesh Functions

Create mesh from .obj file:

In [465]:
def create_mesh(obj_dir, mesh_type):
    surface_mesh = pyvista.read(obj_dir)
    if mesh_type.lower() == 'tetra':
        mindihedral = 20
        vol_mesh = create_volume_mesh(surface_mesh, mindihedral)
    elif mesh_type.lower() == 'hex':
        mindihedral = 1
        vol_mesh = create_volume_mesh(surface_mesh, mindihedral)
        vol_mesh = tet2hex(vol_mesh)
    meshio.write('temp.xdmf', vol_mesh)
    with dolfinx.io.XDMFFile(MPI.COMM_WORLD, 'temp.xdmf', 'r') as f:
        vol_mesh = f.read_mesh(name="Grid")
    for ext in ('.h5', '.xdmf'):
        os.remove('temp'+ext)
    return vol_mesh

Convert Dolfinx mesh to meshio mesh:

In [456]:
def dolfin_to_meshio(dolfin_mesh):
    num_nodes = dolfin_mesh.geometry.dofmap.num_nodes
    cells = dolfin_mesh.geometry.dofmap.array.reshape(num_nodes,-1)
    cell_type = dolfin_mesh.topology.cell_name()
    points = dolfin_mesh.geometry.x
    meshio_mesh = meshio.Mesh(points, {cell_type: cells})
    return meshio_mesh

To create a tetrahedral volumetric mesh from a triangular surface mesh:

In [55]:
def create_volume_mesh(mesh, mindihedral):
    tetgen_mesh = tetgen.TetGen(mesh)
    tetgen_mesh.tetrahedralize(order=1, mindihedral=mindihedral) 
    grid = tetgen_mesh.grid
    pts, cells = grid.points, grid.cells_dict[10]
    mesh = meshio.Mesh(pts, [("tetra", cells)])
    return mesh

To convert a tetraheral volumetric mesh to a hexahedral volumetric mesh:

In [460]:
from itertools import combinations
import numpy as np
from tqdm import tqdm

NUM_VERT=4
NUM_SUBDIV=4
EDGE_COMBOS = list(combinations(range(NUM_VERT),2))
FACE_COMBOS = list(combinations(range(NUM_VERT),3))
EPS = 1e-12
ORDERING_AXIS = 2


EL_0_ORDER = ((  (0,),   (0,1),   (0,1,2),   (0,2)),
               ((0,3), (0,1,3), (0,1,2,3), (0,2,3)))
EL_1_ORDER = ((  (1,),   (1,2),   (0,1,2),   (0,1)),
               ((1,3), (1,2,3), (0,1,2,3), (0,1,3)))
EL_2_ORDER = (( (2,),   (0,2),   (0,1,2),   (1,2)),
              ((2,3), (0,2,3), (0,1,2,3), (1,2,3)))
EL_3_ORDER = (((0,3), (0,1,3), (0,1,2,3), (0,2,3)),
              ( (3,),   (1,3),   (1,2,3),   (2,3)))
SUBDIV_ELS = (EL_0_ORDER, EL_1_ORDER, EL_2_ORDER, EL_3_ORDER)

SUBDIV_VERTS = (*[(i,) for i in range(NUM_VERT)],
                *EDGE_COMBOS,
                *FACE_COMBOS,
                tuple(i for i in range(NUM_VERT)))

SUBDIV_ORDER = []
for el in SUBDIV_ELS:
    face_idx = []
    for f in el:
        vert_idx = []
        for v_1 in f:
            vert_i = [idx for idx, v_2 in enumerate(SUBDIV_VERTS) if v_1==v_2]
            vert_idx.append(vert_i[0])
        face_idx.append(tuple(vert_idx))
    SUBDIV_ORDER.append(tuple(face_idx))
SUBDIV_ORDER = tuple(SUBDIV_ORDER) 

def tet2hex(mesh):
    old_coords = mesh.points
    new_cells, new_coords = np.empty((0,8), int), np.empty((0,3), float)
    for i, verts in enumerate(tqdm(mesh.cells[0].data)):
        # Compute coordinates of four hexahedra which will formed 
        # by dividing up this tetrahedron:
        subdiv_coords = compute_subdiv_coords(verts, old_coords)
        # Update the coordiantes list and cells list:
        new_cells, new_coords = update_cells_and_coords(subdiv_coords, new_cells, new_coords)
    
    new_mesh = meshio.Mesh(new_coords, [("hexahedron", new_cells)])
    return new_mesh

def compute_subdiv_coords(verts, coords):
    vert_coords = np.array([coords[v] for v in verts])
    
    # Order vertex along z axis:
    ordering = np.argsort(vert_coords[:, ORDERING_AXIS])
    vert_coords = vert_coords[ordering]
    
    edge_centres = [np.mean([vert_coords[idx] for idx in edge], axis=0) for edge in EDGE_COMBOS]
    face_centres = [np.mean([vert_coords[idx] for idx in faces], axis=0) for faces in FACE_COMBOS]
    vol_centre = np.mean(vert_coords, axis=0)
    subdiv_coords = np.vstack([*vert_coords, *edge_centres, *face_centres, vol_centre])
    return subdiv_coords

def update_cells_and_coords(hex_coords, new_cells, new_coords):
    
    # First, see if any hex coordinates already exist within our mesh:
    exist_pts = find_exist_pts(hex_coords, new_coords)
    
    # Create new coordinates for those points not currently in the mesh:
    notexist_pts = {}
    num_coords = len(new_coords)
    for i, c in enumerate(hex_coords):
        if i not in exist_pts.keys():
            # Number this new point:
            notexist_pts[i] = num_coords
            # Add new coordinates to coordinates list:
            new_coords = np.vstack([new_coords, c])
            # Note we have one more point in mesh:
            num_coords += 1
    
    global_idx = exist_pts
    global_idx.update(notexist_pts)
    for elem in range(NUM_SUBDIV):
        cell_i = []
        for face in (0, 1):
            cell_i += [global_idx[i] for i in SUBDIV_ORDER[elem][face]]
        new_cells = np.vstack([new_cells, cell_i]) 
    return (new_cells, new_coords)

def find_exist_pts(hex_coords, coords):
    exist_pts = {}
    if coords.size>0:
        for i, c in enumerate(hex_coords):
            coord_dist = abs(coords - c)
            pts_same = np.all(coord_dist<EPS, axis=1)
            if pts_same.sum():
                exist_pts[i] = np.where(pts_same)[0].item()
    return exist_pts

def order_cell_verts(el_idx, cell_i):
    for face in (0, 1):    
        ordering = SUBDIV_ORDER[el_idx][face]

## Visualisation

To visualise a PyVista mesh

In [15]:
def visualise_mesh(grid, subgrid=None, title=None):
    pyvista.start_xvfb(wait=0.05)
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    if subgrid is not None:
        p.add_mesh(grid, style="wireframe", color="k") 
        p.add_mesh(subgrid, color='white', lighting=True, show_edges=True) 
    else: 
        p.add_mesh(grid, show_edges=True, edge_color='k', color='white', lighting=False) # 
    
    p.show_axes()
    # p.show_bounds()
    p.show_grid()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

To extract 'slice' from mesh:

In [16]:
def get_submesh(pyvista_mesh, axis, cutoff):
    cell_center = pyvista_mesh.cell_centers().points
    mask = cell_center[:, axis] < cutoff
    cell_ind = mask.nonzero()[0]
    submesh = pyvista_mesh.extract_cells(cell_ind)
    return submesh

Deformation visualisation:

In [19]:
def plot_deformation(meshio_mesh, uh):
    mesh = meshio_mesh
    pyvista.start_xvfb(wait=0.05)
    topology, cell_types = dolfinx.plot.create_vtk_topology(mesh, mesh.topology.dim)
    grid = pyvista.UnstructuredGrid(topology, cell_types, mesh.geometry.x)
    
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    
    p.add_text("Deformed configuration", name="title", position="upper_edge")
    
    grid["u"] = uh.compute_point_values().real 
    actor_0 = p.add_mesh(grid, style="wireframe", color="k")
    warped = grid.warp_by_vector("u", factor=1.5)
    actor_1 = p.add_mesh(warped)
    
    p.show_axes()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

## Gravitational Loading

Main function to apply gravitational load to mesh:

In [354]:
def apply_loading(obj_dir, y_rot, x_rot, E, nu, rho, g, elem_order, num_steps, mesh_type, u_0=None):
    
    mesh = create_mesh(obj_dir, mesh_type)
    V = dolfinx.VectorFunctionSpace(mesh, ("CG", elem_order))
    
    # Create lambda and mu fields:
    lambda_ = E*nu/((1+nu)*(1-2*nu))
    mu = E/(2*(1+nu))
    
    # Apply fixed BC:
    fixed = lambda x: x[0] < 10
    fixed_facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, fixed)
    facet_tag = dolfinx.MeshTags(mesh, mesh.topology.dim-1, fixed_facets, 1)
    u_bc = dolfinx.Function(V)
    with u_bc.vector.localForm() as loc:
        loc.set(0)
    left_dofs = dolfinx.fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.indices[facet_tag.values==1])
    bcs = [dolfinx.DirichletBC(u_bc, left_dofs)]
    
    B = dolfinx.Constant(mesh, (0, 0, 0))
    T = dolfinx.Constant(mesh, (0, 0, 0))
    v = ufl.TestFunction(V)
        
    u = dolfinx.Function(V)
    d = len(u)
    I = ufl.variable(ufl.Identity(d))
    F = ufl.variable(I + ufl.grad(u))
    C = ufl.variable(F.T * F)
    Ic = ufl.variable(ufl.tr(C))
    J  = ufl.variable(ufl.det(F))
    psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lambda_ / 2) * (ufl.ln(J))**2
    P = ufl.diff(psi, F)
    
    metadata = {"quadrature_degree": elem_order}
    ds = ufl.Measure('ds', subdomain_data=facet_tag, metadata=metadata)
    dx = ufl.Measure("dx", metadata=metadata)
    F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx - ufl.inner(v, T)*ds(2) 
    
    problem = dolfinx.fem.NonlinearProblem(F, u, bcs)
    solver = dolfinx.NewtonSolver(MPI.COMM_WORLD, problem)

    solver.atol = 1e-3
    solver.rtol = 1e-3
    solver.convergence_criterion = "incremental"
    
    g_vector = rho*g*np.array([1,0,0])
    g_vector = rotate_gravity(g_vector, y_rot, x_rot)

    if u_0 is None:
        f_step = g_vector/num_steps
        for n in range(num_steps):
            print(f"Performing load step {n+1}/{num_steps}")
            for i, f_i in enumerate(f_step):
                B.value[i] = (n+1)*f_i
            num_its, converged = solver.solve(u)
            assert(converged)
            u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    else:
        for i, g_i in enumerate(g_vector):
            B.value[i] = g_i
        sol, old_mesh = u_0['sol'], u_0['mesh']
        u_init = interpolate_solution(sol, u._V.tabulate_dof_coordinates(), old_mesh)
        u.vector[:] = u_init.flatten()
        num_its, converged = solver.solve(u)
    return (u, mesh)

Function to rotate gravity using two Euler angles:

In [21]:
# Using Euler angles - see https://www.autonomousrobotslab.com/frame-rotations-and-representations.html
# Here, y_rot = theta, x_rot = psi
ANGLE_TO_RAD = pi/180
def rotate_gravity(g_vector, y_rot, x_rot):
    # NB: Negative associated with y so increasing y_rot goesin 'right direction'
    theta, psi = -ANGLE_TO_RAD*y_rot, ANGLE_TO_RAD*x_rot
    rot_matrix = np.array([[         cos(theta),        0,          -sin(theta)],
                           [sin(psi)*sin(theta),  cos(psi), sin(psi)*cos(theta)],
                           [cos(psi)*sin(theta), -sin(psi), cos(psi)*cos(theta)]])
    rotated_g = rot_matrix @ g_vector
    return rotated_g

Function to interpolate solution at specified points:

In [351]:
def interpolate_solution(sol, x, mesh):
    bb_tree = dolfinx.geometry.BoundingBoxTree(mesh, mesh.topology.dim)
    cells = []
    points_on_proc = []
    for point in x:
        # Find cells that are close to the point
        cell_candidates = dolfinx.geometry.compute_collisions_point(bb_tree, point)
        # Choose one of the cells that contains the point
        cell = dolfinx.geometry.select_colliding_cells(mesh, cell_candidates, point, 1)
        # Only use evaluate for points on current processor
        if len(cell) == 0:
            cell = dolfinx.geometry.compute_closest_entity(bb_tree, point, mesh)
            cell = [cell[0]]
        points_on_proc.append(point)
        cells.append(cell[0])
    points_on_proc = np.array(points_on_proc, dtype=np.float64)
    interp_vals = sol.eval(points_on_proc, cells)
    return interp_vals

# Functions Calls

In [362]:
# Fixed parameters:
elem_order = 2
W = 40 # in mm
L = 90 # in mm
nu = 0.33 # dimensionless
rho = 0.00102 # in g mm^-3
g = 1 # in m s^-2


# Variables:
y_rot = 90
x_rot = -90
E =  100 # in mPa

In [356]:
# Mesh information:
obj_dir = 'breast_tet.obj'
mesh_type = 'tetra'

num_steps = 5
u, mesh = apply_loading(obj_dir, y_rot, x_rot, E, nu, rho, g, elem_order, num_steps, mesh_type)
plot_deformation(mesh, u)

Performing load step 1/5
Performing load step 2/5
Performing load step 3/5
Performing load step 4/5
Performing load step 5/5


VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, color_mappers=[LinearColorMapper(id='104...], height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [464]:
# Mesh information:
obj_dir = 'breast_hex.obj'
mesh_type = 'hex'

num_steps = 5
elem_order = 1
g = 0.01 # in m s^-2
# u_0 = {'sol': u, 'mesh': mesh}
u, mesh = apply_loading(obj_dir, y_rot, x_rot, E, nu, rho, g, elem_order, num_steps, mesh_type)
plot_deformation(mesh, u)

100%|██████████| 481/481 [00:01<00:00, 362.91it/s]


Performing load step 1/5


RuntimeError: Newton solver did not converge because maximum number of iterations reached

In [467]:
mesh = create_mesh('breast_hex.obj', 'hex')
meshio_mesh = dolfin_to_meshio(mesh)
visualise_mesh(meshio_mesh)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [455]:
a.shape

(1924, 8)